In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.1 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
NOTION_TOKEN = "key here"
DATABASE_ID = "20f4296aca078044a20fd88ec8246774"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

# Prompts

In [5]:
#5th prompt
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

2. If the user also wants to "write about ..." in the same request:
    - First, call `notion_page_agent(title=...)`.
    - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
    - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

## EXAMPLE:

If the user says:
"Create a new page named Ichigo. write about anime bleach in my notion page."

You must:

1. Call `notion_page_agent(title='Ichigo')`
2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

---

If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


In [6]:
page_agent_prompt = """
You are a Notion agent responsible for managing pages within a Notion workspace. You're equipped with different tools.

Your primary tasks include:
1. **Creating Pages**: Create new pages with specified titles, content, and properties. Ensure pages are formatted according to user requests.

When performing actions:
- Always ensure page titles and content are clear and concise.
- If tool is not available to perform the action, simply say “I am unable to perform the request"
"""

In [7]:
block_agent_prompt = """
You are a Notion agent responsible for add blocks within Notion pages. Your tasks include:
    - **Creating Blocks**: Add new blocks to a page. Ensure the blocks are created with the proper content and formatting as specified by the user.

2. FORMATTING REQUIREMENTS:
  - Follow Notion's native styling
  - Maintain consistent spacing
  - Preserve hierarchical structure
  - Apply appropriate indentation

## ERROR HANDLING:
  - If a requested block type is unsupported: Respond with ‘Block type not supported’
  - If missing required parameters: Respond with ‘Missing required information: [parameter]’
  - For unauthorized actions: Respond with ‘Permission denied’
  - If tool is not available to perform the action: Respond with ‘I am unable to perform the request’

## Important:
  - When using add_notion_number_list_block, add_notion_bulleted_list_block, and add_notion_to_do_block functions, argument must be a
      list of str. For example
      add_notion_number_list_block([‘item 1°, ‘item 2‘, ‘item 3'])
      add_notion_bulleted_list_block([‘item 1', ‘item 2', ‘item 3'])
      add_notion_to_do_block([‘item 1', ‘item 2', ‘item 3'])
  - When passing a string containing a list of item, Use comma (",") as separator.
    For example, “Chapter 1, Chapter 2, "Chapter 3" is acceptable. “Chapter 1\nChapter 2\n Chapter 3" is not.
"""

# Tools

In [ ]:
import os
from typing import List
import httpx
from langchain_core.tools import tool

NOTION_API_KEY = "key here"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

@tool
def create_notion_page(page_title):
  """
  Create a new page in Notion with the specified title and content.

  args:
      page_title: The title of the new page.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/pages'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  properties= {
        "title": [
            {
                "text": {
                    "content": page_title
                }
            }
        ]
    }
  try:
    response = httpx.post(url, headers=headers, json={
        'parent': {
            'page_id': page_id
        },
        'properties': properties
    })
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}





In [9]:
@tool
def add_notion_heading_block(
        heading_type: str = 'heading_1',
        content: str = 'New Heading',
        is_hyperlink: bool = False,
        hyperlink_url: str = '',
        color: str = 'default'
):
  """
  Add a heading block to a Notion page with the specified content.

  args:
      context_variables: The context variables containing the API key and page ID.
      heading_type: The type of heading block to add (e.g., "heading_1', 'heading_2").
      content: The content of the heading block.
      is_hyperlink: A boolean value indicating whether the heading should be a hyperlink.
      hyperlink_url: The URL of the hyperlink.
      color: The color of the heading block.

  example usage:
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=False)
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=True, hyperlink_url='https://en.wikipedia.org/wiki/Lacinato_kale')
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  text_obj = {'type': 'text', 'text': {'content': content}}
  data = {
      'children': [{
          'object': 'block',
          'type': heading_type,
          heading_type: {
              'rich_text': [text_obj],
              'color': color # Move color inside the heading block
          }
      }]
  }

  if is_hyperlink:
    text_obj['text']['link'] = {'url': hyperlink_url}

  try:
    response = httpx.patch(url, headers=headers, json=data)
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}

In [10]:
@tool
def add_notion_paragraph_block(
    page_id: str,
    content: str,
    is_hyperlink: bool = False,
    hyperlink_url: str = '',
    color: str = 'default'
):
    """
    Add a paragraph block to a Notion page with the specified content.

    Args:
        page_id: The ID of the Notion page where the block will be added.
        content: The content of the paragraph block.
        is_hyperlink: Whether the paragraph is a hyperlink.
        hyperlink_url: The URL if it’s a hyperlink.
        color: The color of the paragraph block.

    Example usage:
        add_notion_paragraph_block(page_id='abc123', content='Hello', is_hyperlink=False)
        add_notion_paragraph_block(page_id='abc123', content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')
    """
    url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
    headers = {
        'Authorization': f'Bearer {NOTION_API_KEY}',
        'Content-Type': 'application/json',
        'Notion-Version': '2022-06-28'
    }

    text_obj = {'type': 'text', 'text': {'content': content}}
    if is_hyperlink:
        text_obj['text']['link'] = {'url': hyperlink_url}

    data = {
        'children': [{
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [text_obj],
                'color': color
            }
        }]
    }

    try:
        response = httpx.patch(url, headers=headers, json=data)
        return response.json()
    except httpx.ReadTimeout:
        return {'error': 'Request timed out'}


In [11]:
@tool
def add_notion_code_block(code: str, Language: str = 'python', caption: List[str] = []):
  """
  Add a code block to a Notion page with the speccified code content.

  args:
    code: The code content of the code block.
    language: The programming language of the code block.
    caption: A list of strings for the caption of the code block.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  data = {
      'children': [{
          'object': 'block',
          'type': 'code',
          'code': {
              'caption': [{'text': {'content': c}} for c in caption],
              'rich_text': [{'type': 'text', 'text': {'content': code}}],
              'language': Language
          }
      }]
  }

  try:
    response = httpx.patch (url, headers-headers, json=data)
    return response.json()
  except httpx. ReadTimeout:
    return {'error': 'Request timed out'}

In [12]:
!pip install -q notion-client

In [13]:
from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."


# Agents with Response

In [14]:
notion_page_agent = create_react_agent(
    model=model,
    tools=[create_notion_page, extract_page_id_from_notion_response],
    prompt=page_agent_prompt,
    name="notion_page_agent"
)

In [15]:
notion_block_agent = create_react_agent(
    model=model,
    tools=[add_notion_heading_block, add_notion_paragraph_block, add_notion_code_block, extract_page_id_from_notion_response],
    prompt=block_agent_prompt,
    name="notion_block_agent"
)

# Supervisor Agent

In [16]:
supervisor = create_supervisor(
    agents=[notion_page_agent, notion_block_agent],

    model=model,
    prompt=delegate_agent_prompt
).compile()



# 5th prompt

In [17]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Shihouin. write 3 sentences about anime character Yoruichi Shihouin in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Shihouin. write 3 sentences about anime character Yoruichi Shihouin in my notion page.', additional_kwargs={}, response_metadata={}, id='cfe436f0-f720-441d-9f6d-c419a1250f79'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mRHkmeyRjGhZrIkdWAIQo1aM', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 447, 'total_tokens': 461, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlXTfqEnsh7vMlIcsVIKR2xuorw48', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='ru

In [22]:
from langchain.schema import HumanMessage

# Define the name of the page and the text content
page_name = "Shihouin12"
text = (
    "Yoruichi Shihouin is a former captain of the 2nd Division in the Gotei 13 and the former commander of the Onmitsukidō. "
    "She is known for her incredible speed and combat skills, earning her the nickname 'Flash Goddess'. "
    "Yoruichi is intelligent, witty, and often appears in her cat form to conceal her true identity."
)

# Stream the message to supervisor
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Create a new page named {page_name}. Write about Yoruichi Shihouin in my Notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")


{'supervisor': {'messages': [HumanMessage(content='Create a new page named Shihouin12. Write about Yoruichi Shihouin in my Notion page.', additional_kwargs={}, response_metadata={}, id='a3fb5faf-fa4a-4641-83aa-11cfe7bb32ea'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kpQkRimvPDmhWMlraNAeyvXA', 'function': {'arguments': '{"title":"Shihouin12"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 444, 'total_tokens': 466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlXeGkTcrMqIzHGaSQ8y1QjxFCKxw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--625

In [19]:
from langchain.schema import HumanMessage

# Define the name of the page and the text content
page_name = "Shihouin12"
text = (
    "Yoruichi Shihouin is a former captain of the 2nd Division in the Gotei 13 and the former commander of the Onmitsukidō. "
    "She is known for her incredible speed and combat skills, earning her the nickname 'Flash Goddess'. "
    "Yoruichi is intelligent, witty, and often appears in her cat form to conceal her true identity."
)

# Stream the message to supervisor
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Create a new page named {page_name}. Write the content in my Notion page:\n\n{text}"
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")


{'supervisor': {'messages': [HumanMessage(content="Create a new page named Shihouin12. Write the content in my Notion page:\n\nYoruichi Shihouin is a former captain of the 2nd Division in the Gotei 13 and the former commander of the Onmitsukidō. She is known for her incredible speed and combat skills, earning her the nickname 'Flash Goddess'. Yoruichi is intelligent, witty, and often appears in her cat form to conceal her true identity.", additional_kwargs={}, response_metadata={}, id='947cfa0c-b76e-4c99-ab80-29e1cec2f1ff'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gA6dxvfMYZ6OzKOTawPomMVl', 'function': {'arguments': '{"title":"Shihouin12"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 514, 'total_tokens': 536, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0